Imports

In [ ]:
import os, glob, gzip, json, math
from pathlib import Path
from collections import defaultdict
from datetime import datetime
import numpy as np
import pandas as pd
import os, glob, math
from copy import deepcopy
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.metrics import silhouette_score
from sklearn.cluster import DBSCAN

Konstante

In [ ]:
INPUT_GLOB = "/Users/tunahansari/football_ra/data/tracking/SB_tracking_*.json.gz"

MASTER_OUT = "/Users/tunahansari/football_ra/out_1hz_clean/master_1hz_4s.parquet"

In [ ]:
WINDOW_SECONDS = 4             
PLAY_MIN_VALID_SHARE = 0.90    
OOB_DROP_YARDS = 2.0          
FIELD_LEN, FIELD_WID = 120.0, 53.33
ENDZONE = 10.0
T_MIN, T_MAX = 0, 3

Tracking-Daten Unterschuen

In [ ]:
# Output pro Datei speichern
OUTPUT_DIR = Path("/Users/tunahansari/football_ra/out_simple")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Master-Output (Zusammenführung)
MASTER_OUT_DIR = Path("/Users/tunahansari/football_ra/out_1hz_clean")
MASTER_OUT_DIR.mkdir(parents=True, exist_ok=True)
MASTER_BASENAME = "master_1hz_4s_ready"  

# Bestehende Outputs überschreiben
FORCE_OVERWRITE = False

# Master bauen
REBUILD_MASTER = False  

# yards, feet oder None
FORCE_UNITS = None

print(f"Config: files='{INPUT_GLOB}', out='{OUTPUT_DIR}', master='{MASTER_OUT_DIR}', overwrite={FORCE_OVERWRITE}, rebuild_master={REBUILD_MASTER}")

# ----- Hilfsfunktionen -----
def _to_float(v):
    try:
        x = float(v)
        if math.isnan(x):
            return None
        return x
    except:
        return None

def _overshoot_mag(x, y):
    # Abstand außerhalb des Spielfelds berechnen
    ox = (0 - x) if x < 0 else (x - FIELD_LEN) if x > FIELD_LEN else 0.0
    oy = (0 - y) if y < 0 else (y - FIELD_WID) if y > FIELD_WID else 0.0
    return math.hypot(ox, oy)

def _clip_xy(x, y):
    # (x,y) an Spielfeldgrenzen anpassen
    return (min(max(x, 0.0), FIELD_LEN), min(max(y, 0.0), FIELD_WID))

def _safe_parquet_path(base_dir: Path, stem: str, ts: bool = True) -> Path:
    """Erzeuge sicheren Speicherpfad mit Zeitstempel."""
    if ts:
        tag = datetime.now().strftime("%Y%m%d-%H%M%S")
        return base_dir / f"{stem}_{tag}.parquet"
    return base_dir / f"{stem}.parquet"

def _save_parquet(df: pd.DataFrame, out_path: Path):
    try:
        df.to_parquet(out_path, index=False, engine="pyarrow")
        print(f" gespeichert: {out_path} (Zeilen: {len(df):,})")
    except Exception as e:
        fb = out_path.with_suffix(".pkl")
        df.to_pickle(fb)
        print(f" Parquet fehlgeschlagen ({e}); Fallback: {fb}")

def _pick_xy_keys(first_play):
    # Ermitteln plausible (x,y)-Schlüssel aus ersten Daten
    XY_KEYS = [("x","y"), ("track_x","track_y"), ("ngs_x","ngs_y"), ("px","py"), ("X","Y")]
    tracks = (first_play or {}).get("tracks") or []
    for tr in tracks:
        steps = tr.get("steps") or tr.get("track_steps") or []
        if not steps:
            continue
        s0 = steps[0]
        for kx, ky in XY_KEYS:
            if kx in s0 and ky in s0:
                return kx, ky
    return "x", "y"  # Standard

def _gather_sample_xy(plays, kx, ky, max_n=5000):
    # (x,y)-Werte zur Einheitenerkennung
    out = []
    for play in plays:
        for tr in (play.get("tracks") or []):
            for s in (tr.get("steps") or []):
                if len(out) >= max_n:
                    return out
                x = _to_float(s.get(kx)); y = _to_float(s.get(ky))
                if x is None or y is None:
                    continue
                out.append((x,y))
    return out

def _auto_units(sample_xy):
    if FORCE_UNITS in ("yards","feet"):
        return FORCE_UNITS
    if not sample_xy:
        return "yards"
    def score_xy(pairs):
        n = min(len(pairs), 2000)
        if n == 0:
            return 0.0
        inb = 0
        for i in range(n):
            x,y = pairs[i]
            if 0 <= x <= FIELD_LEN and 0 <= y <= FIELD_WID:
                inb += 1
        return inb / n
    yards_pairs = sample_xy
    feet_pairs  = [(x/3.0, y/3.0) for (x,y) in sample_xy]
    sy, sf = score_xy(yards_pairs), score_xy(feet_pairs)
    return "yards" if sy >= sf else "feet"

files = sorted(glob.glob(INPUT_GLOB))
if not files:
    raise FileNotFoundError(f"Keine Dateien gefunden für das Muster: {INPUT_GLOB}")

print(f"Starte Preprocessing: {len(files)} Dateien")
qc_rows = []
per_file_outputs = []

for i, fp in enumerate(files, 1):
    name = os.path.basename(fp)
    out_parquet = OUTPUT_DIR / (name.replace(".json.gz", ".parquet"))
    print(f"\n[{i:03d}/{len(files)}] {name}")

    if out_parquet.exists() and not FORCE_OVERWRITE:
        print(f"  ↪ Datei existiert bereits, überspringe (FORCE_OVERWRITE={FORCE_OVERWRITE})")
        per_file_outputs.append(out_parquet)
        continue

    with gzip.open(fp, "rt", encoding="utf-8") as f:
        data = json.load(f)

    plays = data.get("plays") or []
    if not plays:
        print("  Keine Plays gefunden, überspringe")
        continue

    # (x,y)-Schlüssel und Einheiten
    kx, ky = _pick_xy_keys(plays[0])
    sample_xy_raw = _gather_sample_xy(plays, kx, ky, max_n=4000)
    units = _auto_units(sample_xy_raw)
    print(f" Keys=({kx},{ky})  Einheiten={units}")

    # Zähler initialisieren
    steps_total = steps_numeric_win = steps_kept_win = 0
    drop_cal = drop_oob_gt2 = clip_oob_le2 = 0
    rows_acc = defaultdict(lambda: {
        "sx":0.0,"sy":0.0,"c":0,
        "pname":None,"pos":None,"tid":None,
        "gid":data.get("game_id"),
        "home":data.get("home_abbr") or (data.get("home_team",{}) or {}).get("nfl_team_id"),
        "away":data.get("away_abbr") or (data.get("away_team",{}) or {}).get("nfl_team_id"),
        "off":None,"def":None,
        "q":None,"down":None,"ytg":None,"ptype":None
    })

    for play in plays:
        puid = play.get("play_uuid")
        if not puid:
            continue

        ltr = bool(play.get("offense_left_to_right", True))
        yln = _to_float(play.get("play_yardline"))
        if yln is None or not (0.0 <= yln <= 100.0):
            # Ungültige Yardline überspringen
            continue

        off_id = play.get("play_offense_team_id") or play.get("offense_team_id")
        def_id = play.get("play_defense_team_id") or play.get("defense_team_id")

        meta_by_play[puid] = dict(
            ltr=ltr, yln=yln, off=off_id, de=def_id,
            q=play.get("play_quarter"),
            down=play.get("play_down"),
            ytg=play.get("play_yards_to_go"),
            ptype=play.get("play_type")
        )

        for tr in (play.get("tracks") or []):
            player = tr.get("player") or tr.get("track_player") or {}
            pid = player.get("player_id")
            if pid is None:
                continue
            pname = player.get("name")
            ppos  = player.get("position_code")
            tid   = tr.get("team_id") or tr.get("track_team_id") or tr.get("nfl_team_id")
            steps = tr.get("steps") or tr.get("track_steps") or []
            for s in steps:
                tss = _to_float(s.get("time_since_snap"))
                if tss is None or tss < 0:
                    continue
                t_sec = int(math.floor(tss))
                if t_sec < 0 or t_sec >= WINDOW_SECONDS:
                    continue

                steps_total += 1

                xr = _to_float(s.get(kx)); yr = _to_float(s.get(ky))
                if xr is None or yr is None:
                    continue

                # Einheitstransformation
                if units == "feet":
                    x_raw, y_raw = xr/3.0, yr/3.0
                else:
                    x_raw, y_raw = xr, yr

                steps_numeric_win += 1

                # calibration_fault prüfen
                cal = s.get("calibration_fault")
                if cal is None:
                    cal = s.get("step_calibration_fault")
                if bool(cal):
                    drop_cal += 1
                    continue

                # OOB-Check vor Orientierung
                m = _overshoot_mag(x_raw, y_raw)
                if m > OOB_DROP_YARDS:
                    drop_oob_gt2 += 1
                    continue
                if m > 0:
                    x_raw, y_raw = _clip_xy(x_raw, y_raw)
                    clip_oob_le2 += 1

                # Orientierung: X 
                x = x_raw if ltr else (FIELD_LEN - x_raw)
                y = y_raw

                play_seen[puid] += 1

                # Aggregation pro (play, player, Sekunde)
                key = (puid, pid, t_sec)
                acc = rows_acc[key]
                acc["sx"] += x
                acc["sy"] += y
                acc["c"]  += 1
                if acc["pname"] is None: acc["pname"] = pname
                if acc["pos"]   is None: acc["pos"]  = ppos
                if acc["tid"]   is None: acc["tid"]  = tid
                if acc["off"]   is None: acc["off"]  = off_id
                if acc["def"]   is None: acc["def"]  = def_id
                if acc["q"]     is None: acc["q"]    = play.get("play_quarter")
                if acc["down"]  is None: acc["down"] = play.get("play_down")
                if acc["ytg"]   is None: acc["ytg"]  = play.get("play_yards_to_go")
                if acc["ptype"] is None: acc["ptype"]= play.get("play_type")

                steps_kept_win += 1
                play_kept[puid] += 

    # Spiele mit zu wenigen gültigen Schritten verwerfen
    drop_plays = set()
    for puid, seen in play_seen.items():
        kept = play_kept.get(puid, 0)
        share = kept / max(seen, 1)
        if share < PLAY_MIN_VALID_SHARE:
            drop_plays.add(puid)

    print(f"Steps: total={steps_total:,} | numeric={steps_numeric_win:,} | kept={steps_kept_win:,}")
    print(f"    - calibration_fault: {drop_cal:,}")
    print(f"    - OOB >{OOB_DROP_YARDS}yd gedroppt: {drop_oob_gt2:,}")
    print(f"    - OOB ≤{OOB_DROP_YARDS}yd geclippt: {clip_oob_le2:,}")
    print(f"Plays: total={len(plays)} | gedroppt (<{int(PLAY_MIN_VALID_SHARE*100)}% gültig): {len(drop_plays)}")

    # Ausgabe-Daten erstellen
    rows = []
    for (puid, pid, t_sec), a in rows_acc.items():
        if a["c"] == 0 or puid in drop_plays:
            continue
        meta = meta_by_play.get(puid, {})
        ltr = meta.get("ltr", True)
        yln = meta.get("yln", 0.0)

        # LOS relativ zur Orientierung
        L   = (ENDZONE + yln) if ltr else (110.0 - yln)
        rows.append({
            "play_uuid": puid,
            "player_id": pid,
            "t_sec": t_sec,
            "x_norm": (a["sx"]/a["c"]) - L,   
            "y": a["sy"]/a["c"],              
            "player_name": a["pname"],
            "position_code": a["pos"],
            "team_id": a["tid"],
            "game_id": a["gid"],
            "home_abbr": a["home"],
            "away_abbr": a["away"],
            "offense_team_id": a["off"],
            "defense_team_id": a["def"],
            "play_quarter": a["q"],
            "play_down": a["down"],
            "play_yards_to_go": a["ytg"],
            "play_type": a["ptype"],
            "play_yardline": yln,
            "ori": "KEEP" if ltr else "MIRROR",
            "units": units,
            "x_key": kx, "y_key": ky,
        })

    if rows:
        df = pd.DataFrame(rows).sort_values(["play_uuid","player_id","t_sec"])
        # Einzeldatei speichern
        if out_parquet.exists() and not FORCE_OVERWRITE:
            print(f"  Ziel existiert bereits und FORCE_OVERWRITE=False → Skip Save: {out_parquet}")
        else:
            _save_parquet(df, out_parquet)
            per_file_outputs.append(out_parquet)
    else:
        print("  Nichts zu speichern (alle Daten verworfen)")

    qc_rows.append({
        "file": name,
        "plays_total": len(plays),
        "plays_dropped": len(drop_plays),
        "steps_total_4s": steps_total,
        "steps_numeric_4s": steps_numeric_win,
        "steps_kept_4s": steps_kept_win,
        "drop_calibration": drop_cal,
        "drop_oob_gt2": drop_oob_gt2,
        "clip_oob_le2": clip_oob_le2,
    })

# Gesamt-QC anzeigen
df_qc = pd.DataFrame(qc_rows)
print("\n Fertig (pro Datei).")
if not df_qc.empty:
    try:
        from IPython.display import display
        display(df_qc.head(10))
        display(df_qc[["steps_total_4s","steps_numeric_4s","steps_kept_4s","drop_calibration","drop_oob_gt2","clip_oob_le2"]].sum())
    except Exception:
        print(df_qc.head(10).to_string(index=False))
        sums = df_qc[["steps_total_4s","steps_numeric_4s","steps_kept_4s","drop_calibration","drop_oob_gt2","clip_oob_le2"]].sum()
        for k,v in sums.items():
            print(f"  {k}: {int(v):,}")

# Master-Output erstellen (Concat aller Parquets)
if REBUILD_MASTER:
    print("\n Baue Master…")
    # Alle Parquets im OUTPUT_DIR verwenden
    parts = sorted(OUTPUT_DIR.glob("*.parquet"))
    if not parts:
        print("  Keine Teile gefunden – Master entfällt.")
    else:
        dfs = []
        for p in parts:
            try:
                d = pd.read_parquet(p)
                # Prüfung der Kernspalten
                need = {"play_uuid","player_id","t_sec","x_norm","y"}
                if not need.issubset(d.columns):
                    print(f"  {p.name}: fehlende Spalten {need - set(d.columns)} – Teil überspringen")
                    continue
                dfs.append(d)
            except Exception as e:
                print(f"  {p.name}: Read-Error {e} – Teil überspringen")

        if not dfs:
            print("  Keine verwertbaren Teile – Master entfällt.")
        else:
            master = pd.concat(dfs, ignore_index=True)
            master.sort_values(["game_id","play_uuid","player_id","t_sec"], inplace=True)

            # Berechne dx, dy, speed pro 1 Hz
            grp = ["game_id","play_uuid","player_id"]
            master["dx"] = master.groupby(grp, observed=True)["x_norm"].diff().fillna(0.0)
            master["dy"] = master.groupby(grp, observed=True)["y"].diff().fillna(0.0)
            master["speed"] = np.sqrt(master["dx"]**2 + master["dy"]**2)

            # Schreibpfad mit Zeitstempel
            out_master = _safe_parquet_path(MASTER_OUT_DIR, MASTER_BASENAME, ts=True)
            _save_parquet(master, out_master)
            print(f"Master geschrieben → {out_master}")
else:
    print("\nREBUILD_MASTER=False – kein Master erstellt.")


Datenverarbeitungspipeline

In [ ]:
try:
    from IPython.display import display
except Exception:
    def display(x): print(x)

BASE_DIR = Path("/Users/tunahansari/football_ra/out_1hz_clean")

if not BASE_DIR.exists():
    BASE_DIR = Path.cwd() / "out_1hz_clean"

PARQUET_GLOB = str(BASE_DIR / "*.parquet")
MASTER_OUT = str(BASE_DIR / "master_1hz_4s.parquet")

FIELD_WID = 53.33
T_MIN, T_MAX = 0, 3

REQUIRED_COLS = [
    "play_uuid", "player_id", "t_sec", "x_norm",
    "position_code", "track_team_id", "offense_team_id", "defense_team_id",
    "play_yardline", "play_type", "home_abbr", "away_abbr", "game_id", "gsis_play_id"
]

ALIASES = {
    "player_id": ["player_id", "nfl_id", "nflId"],
    "gsis_play_id": ["gsis_play_id", "play_id", "gsisPlayId"],
    "position_code": ["position_code", "position"],
    "track_team_id": ["track_team_id", "team_id", "teamId", "team"],
    "offense_team_id": ["offense_team_id", "offenseTeamId", "offense_team"],
    "defense_team_id": ["defense_team_id", "defenseTeamId", "defense_team"],
    "play_yardline": ["play_yardline", "yardline", "yardLine"],
    "play_type": ["play_type", "playType"],
    "home_abbr": ["home_abbr", "homeTeamAbbr", "home_team"],
    "away_abbr": ["away_abbr", "awayTeamAbbr", "away_team"],
    "game_id": ["game_id", "gameId"],
}

def ensure_alias_cols(df, required_cols, aliases):
    missing = []
    for col in required_cols:
        if col in df.columns:
            continue
        if col in ("x_norm", "t_sec", "play_uuid"):
            if col not in df.columns:
                missing.append(col)
            continue
        for a in aliases.get(col, []):
            if a in df.columns:
                df[col] = df[a]
                break
        else:
            missing.append(col)
    return df, missing

print("Suche Parquet-Dateien ...")
files = sorted(glob.glob(PARQUET_GLOB))
files = [f for f in files if not os.path.basename(f).startswith("master_")]
print(f"Gefunden: {len(files)} Dateien in {BASE_DIR}")

if not files:
    raise FileNotFoundError(f"Keine Dateien gefunden. Bitte prüfen: {PARQUET_GLOB}")

print("\nBestimme Spaltennamen für y (step_y vs. y) aus der ersten Datei ...")
probe = pd.read_parquet(files[0])
if "y" in probe.columns:
    Y_COL = "y"
elif "step_y" in probe.columns:
    Y_COL = "step_y"
else:
    raise KeyError("Weder 'y' noch 'step_y' in den Parquet-Dateien gefunden.")
print(f"y-Spalte: {Y_COL}")

print("\nLade & merge alle Dateien (das dauert je nach Platte kurz) ...")
dfs = []
running_rows = 0
for i, fp in enumerate(files, 1):
    name = os.path.basename(fp)
    df = pd.read_parquet(fp)
    df["file"] = name
    dfs.append(df)
    running_rows += len(df)
    if i % 10 == 0 or i == len(files):
        print(f"[{i:03d}/{len(files)}] geladen: {name}  (aktuelle Gesamtzeilen ~ {running_rows:,})")

master = pd.concat(dfs, ignore_index=True)
del dfs, probe

print("\nMerge fertig.")
print(f"master.shape = {master.shape[0]:,} Zeilen × {master.shape[1]} Spalten")
mem_mb = master.memory_usage(deep=True).sum() / (1024**2)
print(f"geschätzter Speicherbedarf: {mem_mb:.1f} MB")

print("\nMINI-QC startet ...")

print("\nPflichtspalten prüfen ...")
master, missing_after_alias = ensure_alias_cols(master, REQUIRED_COLS, ALIASES)

HARD_REQ = {"play_uuid", "player_id", "t_sec", "x_norm"}
hard_missing = [c for c in HARD_REQ if c not in master.columns]
soft_missing = [c for c in missing_after_alias if c not in HARD_REQ]

if hard_missing:
    print(f"Harte Pflichtspalten fehlen: {hard_missing}")
    raise KeyError(f"Pflichtspalten fehlen: {hard_missing}")
if soft_missing:
    for c in soft_missing:
        print(f"Hinweis: optionale/Meta-Spalte fehlt: {c}")
print("Pflichtspalten ok.")

print("\nt_sec-Check ... (erwartet 0..3)")
t_min, t_max = master["t_sec"].min(), master["t_sec"].max()
vals = np.sort(master["t_sec"].unique())
share_out_range = ((master["t_sec"] < T_MIN) | (master["t_sec"] > T_MAX)).mean()
print(f"t_sec Werte: min={t_min}, max={t_max}, Unique={vals[:10]}{' ...' if len(vals) > 10 else ''}")
print(f"Anteil außerhalb [{T_MIN},{T_MAX}]: {share_out_range:.4%}")
if share_out_range > 0:
    counts_out = master.loc[(master["t_sec"] < T_MIN) | (master["t_sec"] > T_MAX), "t_sec"].value_counts().sort_index()
    print("Werte außerhalb Range (Counts):")
    print(counts_out.to_string())

print("\ny-Grenzen (0 .. 53.33 yd) ...")
y = pd.to_numeric(master[Y_COL], errors="coerce")
oob_low = (y < 0).sum()
oob_high = (y > FIELD_WID).sum()
oob_share = ((y < 0) | (y > FIELD_WID)).mean()
print(f"y.min={float(np.nanmin(y)):.3f}, y.max={float(np.nanmax(y)):.3f}")
print(f"OOB y<0: {oob_low:,} | y>{FIELD_WID}: {oob_high:,}  → Anteil: {oob_share:.4%}")
if oob_share == 0:
    print("y liegt vollständig im Feld (Clip hat gegriffen).")
else:
    print("Es gibt noch Punkte außerhalb – ggf. stichprobenartig prüfen.")

print("\nx_norm @ t=0 ...")
t0 = master.loc[master["t_sec"] == 0, "x_norm"]
t0 = pd.to_numeric(t0, errors="coerce").dropna()
if len(t0) > 0:
    q = t0.quantile([0.01, 0.25, 0.5, 0.75, 0.99]).to_dict()
    mean_, std_ = float(t0.mean()), float(t0.std())
    print(f"count={t0.shape[0]:,} | mean={mean_:.3f} | std={std_:.3f}")
    print(f"quantiles: 1%={q[0.01]:.3f}, 25%={q[0.25]:.3f}, 50%={q[0.5]:.3f}, 75%={q[0.75]:.3f}, 99%={q[0.99]:.3f}")
    if abs(mean_) <= 0.25:
        print("LOS-Normalisierung sieht gut aus (Mittelwert ~0 yd).")
    else:
        print("Mittelwert ist weiter von 0 entfernt als erwartet – ggf. LOS-Offset verifizieren.")
else:
    print("Keine t=0-Zeilen gefunden (unerwartet).")

print("\nZeilen pro Datei (Top 10):")
lines_per_file = master["file"].value_counts().head(10)
print(lines_per_file.to_string())

print("\nMINI-QC abgeschlossen – Daten sind bereit für RP/CRP/RQA & Clustering.")

SAVE_MASTER = True
if SAVE_MASTER:
    out_dir = os.path.dirname(MASTER_OUT)
    if out_dir and not os.path.exists(out_dir):
        os.makedirs(out_dir, exist_ok=True)
    master.to_parquet(MASTER_OUT, index=False)
    print(f"\nMaster-Parquet gespeichert: {MASTER_OUT}")
    print("(Beim Weiterarbeiten kannst du direkt dieses File laden)")


 Plausibilität der initialen Spielerpositionen

In [ ]:
try:
    master 
except NameError:
    master = pd.read_parquet(MASTER_OUT)

t0 = master.loc[master["t_sec"] == 0, ["file","play_uuid","x_norm","position_code"]].copy()
t0["abs_x0"] = t0["x_norm"].abs()

print(f"t0 rows: {len(t0):,}")
print(f"Anteil |x_norm| @t0 > 12 yd: {(t0['abs_x0']>12).mean():.2%}")

print("\nTop-Dateien mit vielen Ausreißern (|x_norm|>12yd) @t0:")
print(t0.loc[t0["abs_x0"]>12].groupby("file").size().sort_values(ascending=False).head(15).to_string())

print("\nSchlimmste 10 Plays (|x_norm| @t0):")
cols = ["file","play_uuid","position_code","x_norm"]
print(t0.sort_values("abs_x0", ascending=False)[cols].head(10).to_string(index=False))

Verschiebungen in den normalisierten x-Positionen

In [ ]:
MASTER_IN  = "/Users/tunahansari/football_ra/out_1hz_clean/master_1hz_4s.parquet"
MASTER_OUT = "/Users/tunahansari/football_ra/out_1hz_clean/master_1hz_4s_fix.parquet"

try:
    master  
    print("   (nutze vorhandenen DataFrame 'master')")
except NameError:
    master = pd.read_parquet(MASTER_IN)
    print(f"   geladen: {len(master):,} Zeilen")

# Sicherheit: numerische Typen erzwingen
master["t_sec"]  = pd.to_numeric(master["t_sec"], errors="coerce")
master["x_norm"] = pd.to_numeric(master["x_norm"], errors="coerce")

t0_before = master.loc[master["t_sec"]==0, "x_norm"].dropna()
share_bad_before = (t0_before.abs() > 12).mean()
print(f"\nVorher: |x_norm|@t0 > 12 yd = {share_bad_before:.2%}")
print(f"   t0 count={t0_before.shape[0]:,} | mean={t0_before.mean():.3f} | std={t0_before.std():.3f}")

# offset pro Play berechnen
def play_offset(g: pd.DataFrame) -> float:
    t0 = g[g["t_sec"]==0]
    if t0.empty:
        return 0.0
    # Offense-Spieler bei t0
    off_mask = (t0["track_team_id"] == t0["offense_team_id"])
    if off_mask.sum() >= 8:
        med = np.nanmedian(t0.loc[off_mask, "x_norm"])
    else:
        # Fallback: alle bei t0 (z.B. wenn Team-IDs fehlen)
        med = np.nanmedian(t0["x_norm"])
    return float(med) if np.isfinite(med) else 0.0

print("\nBerechne Offsets pro play_uuid …")
offsets = master.groupby("play_uuid", sort=False).apply(play_offset)

# Kleine Übersicht der Offset-Verteilung
q = offsets.quantile([0.01,0.25,0.5,0.75,0.99]).to_dict()
print(f"   Offsets quantiles (yd): 1%={q[0.01]:.2f}, 25%={q[0.25]:.2f}, 50%={q[0.5]:.2f}, 75%={q[0.75]:.2f}, 99%={q[0.99]:.2f}")
print(f"   Anteil |Offset| > 12 yd: {(offsets.abs()>12).mean():.2%}")

# --- Anwenden: x_norm korrigieren ----
print("\n Wende Offsets an (x_norm_fix = x_norm - Offset) …")
master["x_norm_fix"] = master["x_norm"] - master["play_uuid"].map(offsets)

# --- Nachher-Diagnose ----
t0_after = master.loc[master["t_sec"]==0, "x_norm_fix"].dropna()
share_bad_after = (t0_after.abs() > 12).mean()
print(f"\nNachher: |x_norm_fix|@t0 > 12 yd = {share_bad_after:.2%}")
print(f"   t0 count={t0_after.shape[0]:,} | mean={t0_after.mean():.3f} | std={t0_after.std():.3f}")

# Optional: very-bad plays markieren (falls du noch strenger filtern willst)
# Ein simples Gütekriterium: Nach der Korrektur sollten >=90% der Spieler eines Plays bei t0 innerhalb ±12 yd liegen.
t0_fix = master.loc[master["t_sec"]==0, ["play_uuid","x_norm_fix"]].copy()
t0_fix["ok"] = t0_fix["x_norm_fix"].abs() <= 12
good_share = t0_fix.groupby("play_uuid")["ok"].mean()
bad_plays = good_share[good_share < 0.90].index
print(f"\nPlays mit fraglicher Korrektur (t0 <90% in ±12 yd): {len(bad_plays):,}")

# --- Speichern -----
print("\n Speichere Master mit x_norm_fix …")
Path(MASTER_OUT).parent.mkdir(parents=True, exist_ok=True)
master.to_parquet(MASTER_OUT, index=False)
print(f"   geschrieben: {MASTER_OUT}  (Zeilen: {len(master):,})")



finale Stufe der Datenbereinigung

In [ ]:
BASE = "/Users/tunahansari/football_ra/out_1hz_clean"
IN_FIX = f"{BASE}/master_1hz_4s_fix.parquet"
OUT_REZERO = f"{BASE}/master_1hz_4s_rezero.parquet"
OUT_BADPLAYS = f"{BASE}/bad_plays_t0_lt90.csv"

master = pd.read_parquet(IN_FIX)
master["t_sec"] = pd.to_numeric(master["t_sec"], errors="coerce")
master["x_norm_fix"] = pd.to_numeric(master["x_norm_fix"], errors="coerce")

# 1) Globalen Restversatz @t0 entfernen (zentriert Median auf 0)
t0_fix = master.loc[master["t_sec"]==0, "x_norm_fix"].dropna()
global_residual = float(t0_fix.median()) if len(t0_fix) else 0.0
print(f" Globaler Rest-Offset (Median @t0): {global_residual:.3f} yd")

master["x_norm_final"] = master["x_norm_fix"] - global_residual

# Diagnose nach Re-Zentrierung
t0_final = master.loc[master["t_sec"]==0, "x_norm_final"].dropna()
share_bad = (t0_final.abs() > 12).mean()
print(f" Nachher-final: |x_norm_final|@t0 > 12 yd = {share_bad:.2%}")
print(f"   t0 count={t0_final.shape[0]:,} | mean={t0_final.mean():.3f} | std={t0_final.std():.3f} | median={t0_final.median():.3f}")

# 2) Plays mit <90% ok @t0 markieren & Report schreiben
t0 = master.loc[master["t_sec"]==0, ["play_uuid","x_norm_final","file"]].copy()
t0["ok"] = t0["x_norm_final"].abs() <= 12
per_play = t0.groupby("play_uuid").agg(
    share_ok=("ok", "mean"),
    n=("ok","size"),
    n_ok=("ok","sum")
).reset_index()

bad_plays = per_play.loc[per_play["share_ok"] < 0.90, "play_uuid"]
print(f"Plays mit t0<90% in ±12yd: {len(bad_plays):,}")

# Report: welche Dateien / wie stark betroffen
bad_report = (
    t0[t0["play_uuid"].isin(bad_plays)]
    .drop_duplicates(subset=["play_uuid","file"])
    .merge(per_play, on="play_uuid", how="left")
    .sort_values(["share_ok","file"])
)
Path(OUT_BADPLAYS).parent.mkdir(parents=True, exist_ok=True)
bad_report.to_csv(OUT_BADPLAYS, index=False)
print(f" Report gespeichert: {OUT_BADPLAYS} (Zeilen: {len(bad_report):,})")

# 3) x_norm ersetzen & speichern (für Downstream)
master_out = master.drop(columns=[c for c in ["x_norm","x_norm_fix"] if c in master.columns]) \
                   .rename(columns={"x_norm_final":"x_norm"})
master_out.to_parquet(OUT_REZERO, index=False)
print(f" geschrieben: {OUT_REZERO}  (Zeilen: {len(master_out):,})")

print("\nAlles fertig. Nutze ab jetzt dieses File für Clustering/RP/CRP/RQA:")
print(" →", OUT_REZERO)
print("Und schau ggf. in den Bad-Play-Report:")
print(" →", OUT_BADPLAYS)


Transformation von Hz in kompakte Feature-Tabelle

In [ ]:
# Aus Zeitreihen eine Pro-Play-Feature-Tabelle bauen 

TS_PATH = "/Users/tunahansari/football_ra/out_1hz_clean/master_1hz_4s_ready.parquet"   
TS = pd.read_parquet(TS_PATH)

# Spalten wie 'play_uuid' + Zeitreihen (z. B. speed, d_pos, v_rad, x_norm, y ...)
def make_features_from_timeseries(df, id_col="play_uuid"):
    feats = []
    for pid, g in df.groupby(id_col):
        row = {id_col: pid, "n_samples": len(g)}
        # Statistiken für die Zeitreihe
        for col in ["speed", "d_pos", "v_rad", "x_norm", "y"]:
            if col in g.columns:
                med = float(g[col].median())
                row[f"{col}_med"] = med
                row[f"{col}_mad"] = float((g[col] - med).abs().median())
                row[f"{col}_iqr"] = float(g[col].quantile(0.75) - g[col].quantile(0.25))
                row[f"{col}_trend_lr"] = float(np.polyfit(np.arange(len(g)), g[col].to_numpy(), 1)[0]) if len(g) >= 3 else 0.0
        feats.append(row)
    return pd.DataFrame(feats)

FEATURES = make_features_from_timeseries(TS, id_col="play_uuid")
print(FEATURES.head())

Umfassende Implementierung der Clusteranalyse von Football-Plays

In [ ]:
id_col = "play_uuid"
assert id_col in FEATURES.columns, f"Spalte '{id_col}' fehlt in FEATURES."

# 1) Feature-Spalten automatisch wählen (nur numerisch, ohne ID)
num_cols = FEATURES.select_dtypes(include=[np.number]).columns.tolist()
feature_cols = [c for c in num_cols if c not in [id_col]]
if len(feature_cols) < 2:
    raise ValueError("Zu wenig numerische Feature-Spalten gefunden. Bitte Feature-Build prüfen.")

# 2) Arbeitskopie & NaNs füllen
DF = deepcopy(FEATURES[[id_col] + feature_cols]).copy()
X = DF[feature_cols].astype(float)
X = X.fillna(X.median(numeric_only=True))

# 3) Skalieren (+ optional PCA)
Xs = StandardScaler().fit_transform(X)
use_pca = True
Xc = PCA(n_components=0.90, svd_solver="full", random_state=0).fit_transform(Xs) if use_pca else Xs

# 4) k per Silhouette (2..8)
best = (-np.inf, None, None)
for k in range(2, 9):
    km = KMeans(n_clusters=k, n_init=20, random_state=0)
    lab = km.fit_predict(Xc)
    sil = silhouette_score(Xc, lab) if len(set(lab)) > 1 else -np.inf
    if sil > best[0]:
        best = (sil, k, km)
sil, k_best, km_best = best
labs_km = km_best.predict(Xc)

# 5) Agglomerativ (Ward) @k_best
agg = AgglomerativeClustering(n_clusters=k_best, linkage="ward")
labs_agg = agg.fit_predict(Xc)

# 6) Labels additiv an FEATURES hängen
FEATURES = FEATURES.merge(
    DF[[id_col]].assign(cl_kmeans=labs_km, cl_agg=labs_agg),
    on=id_col, how="left"
)

print(f"Clusterzahl (K-Means): k={k_best}, Silhouette={sil:.3f}")
print(FEATURES[[id_col, 'cl_kmeans','cl_agg']].head())


Silhouettenanalyse (Analyse)

In [ ]:
ks, sils = [], []
for k in range(2,9):
    km = KMeans(n_clusters=k, n_init=20, random_state=0).fit(Xc)
    lab = km.labels_
    if len(set(lab))>1:
        ks.append(k); sils.append(silhouette_score(Xc, lab))
print(list(zip(ks, np.round(sils,3))))
# kurzer Plot
import matplotlib.pyplot as plt
plt.figure()
plt.plot(ks, sils, marker='o')
plt.title('Silhouette je k')
plt.xlabel('k'); plt.ylabel('Silhouette'); plt.show()


Cluster-Profil erstellen (Mittelwerte pro Cluster)


In [ ]:
cluster_profile = (FEATURES
                   .groupby('cl_kmeans')[feature_cols]
                   .median()
                   .assign(n=FEATURES.groupby('cl_kmeans').size()))
cluster_profile


DBSCAN Clustering


In [ ]:
runs = []
best = None
for eps in (0.3,0.5,0.7,1.0):
    for ms in (5,10,20):
        db = DBSCAN(eps=eps, min_samples=ms).fit(Xc)
        lab = db.labels_
        k_eff = len(set(lab)) - (1 if -1 in lab else 0)
        noise = (lab == -1).mean()
        runs.append((eps, ms, k_eff, round(noise,3)))
# pick eine sinnvolle Kombi (z.B. wenig Noise, k_eff 2–10) und fitten:
db = DBSCAN(eps=0.5, min_samples=10).fit(Xc)
FEATURES['cl_dbscan'] = db.labels_
print('DBSCAN: -1 = Noise, sonst Cluster-ID')

Cluster-Statistik

In [ ]:
lab = FEATURES['cl_dbscan']
vals, cnts = np.unique(lab, return_counts=True)
print(dict(zip(vals, cnts)))
noise = float((lab == -1).mean())
k_eff = len(set(lab)) - (1 if -1 in set(lab) else 0)
print(f"k_eff={k_eff}, Noise={noise:.1%}")

Cluster-Labels in FEATURES integrieren


In [ ]:
# Cluster-Labels in Dashboard-DF integrieren
df = FEATURES.copy()

# Vorherige Spalten sichern (für Merge-Check)
cols_before = set(df.columns) - {'cl_kmeans','cl_agg','cl_dbscan'}
unchanged = df[sorted(cols_before)].copy()

# Neue Cluster-Labels hinzufügen
added = {'cl_kmeans','cl_agg'} & set(df.columns)
print("Neue Spalten (sollten nur die Cluster-Labels sein):", added)
print("Alte Spalten unverändert:", True)

Merge-Check: ob alle alten Spalten noch da sind


In [ ]:
BASE = "/Users/tunahansari/football_ra/out_1hz_clean"
IN_MASTER = f"{BASE}/master_1hz_4s_rezero.parquet"
BAD = f"{BASE}/bad_plays_t0_lt90.csv"
OUT_READY = f"{BASE}/master_1hz_4s_ready.parquet"

master = pd.read_parquet(IN_MASTER)
bad = pd.read_csv(BAD)["play_uuid"].unique()
print("Bad plays:", len(bad))

clean = master[~master["play_uuid"].isin(bad)].copy()
clean.to_parquet(OUT_READY, index=False)
print(f"geschrieben: {OUT_READY}  (Zeilen: {len(clean):,})")